<a href="https://colab.research.google.com/github/ravindersturna/Baseball-Player-Salary-Model/blob/main/DissCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 1000JD.csv to 1000JD.csv


In [3]:
import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['1000JD.csv']))

In [4]:
data.shape

(1015, 14)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   country          1015 non-null   object
 1   country_code     1015 non-null   object
 2   date_added       6 non-null      object
 3   has_expired      1015 non-null   object
 4   job_board        1015 non-null   object
 5   job_description  1015 non-null   object
 6   job_title        1015 non-null   object
 7   job_type         944 non-null    object
 8   location         1015 non-null   object
 9   organization     558 non-null    object
 10  page_url         1015 non-null   object
 11  salary           118 non-null    object
 12  sector           831 non-null    object
 13  uniq_id          1015 non-null   object
dtypes: object(14)
memory usage: 111.1+ KB


In [6]:
data.head()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If youâ€™re considering a care...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID#Â 76162 # PositionsÂ 1 StateÂ C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


In [7]:
sample = data.head(100)

In [8]:
sample.head()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If youâ€™re considering a care...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID#Â 76162 # PositionsÂ 1 StateÂ C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


In [9]:
# Remove unwanted columns
df = sample.drop(columns=['job_title','country','country_code','date_added','has_expired','job_board','job_type','location','organization','page_url','salary','sector','uniq_id'])

In [10]:
df.head()

,job_description
0,TeamSoft is seeing an IT Support Specialist to...
1,The Wisconsin State Journal is seeking a flexi...
2,Report this job About the Job DePuy Synthes Co...
3,Why Join Altec? If youâ€™re considering a care...
4,Position ID#Â 76162 # PositionsÂ 1 StateÂ C...


In [11]:
print('Text Preprocessing and Cleaning')

Text Preprocessing and Cleaning


In [12]:
# Lowercase all text
df['job_description'] = df['job_description'].str.lower()

In [13]:
df.head()

,job_description
0,teamsoft is seeing an it support specialist to...
1,the wisconsin state journal is seeking a flexi...
2,report this job about the job depuy synthes co...
3,why join altec? if youâ€™re considering a care...
4,position id#â 76162 # positionsâ 1 stateâ c...


In [14]:
pip install nltk

In [15]:
# Import stopwords with nltk.
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
new_stopwords = ["to", "on", "#", "@", "&", "?", "*", "â", "€"]

stpwrd = stopwords.words('english')

stpwrd.extend(new_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
# Remove Stopwords
df['clean'] = df['job_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrd)]))

In [17]:
# Remove Punctuations
df["clean"] = df['clean'].str.replace('[^\w\s]','')

In [18]:
df.head()

,job_description,clean
0,teamsoft is seeing an it support specialist to...,teamsoft seeing support specialist join client...
1,the wisconsin state journal is seeking a flexi...,wisconsin state journal seeking flexible motiv...
2,report this job about the job depuy synthes co...,report job job depuy synthes companies member ...
3,why join altec? if youâ€™re considering a care...,join altec youâre considering career altec inc...
4,position id#â 76162 # positionsâ 1 stateâ c...,position idâ 76162 positionsâ 1 stateâ ct city...


In [19]:
pip install yake

     |████████████████████████████████| 60 kB 5.5 MB/s 
     |████████████████████████████████| 134 kB 24.7 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.8.8-cp37-cp37m-linux_x86_64.whl size=73220 sha256=ab04552d25c43371146109e79a7fdcf60fcf5d6caebcb216e2af866229ee490c
  Stored in directory: /root/.cache/pip/wheels/82/aa/f4/716387e1f167cbbf911488aa056138152f4d8699c9c9b43ea8
  Created wheel for segtok: filename=segtok-1.5.10-py3-none-any.whl size=25030 sha256=5f5ce5719860207c0f9059e0742cb3520a5f41c4cd0928b0c12435661a27abdd
  Stored in directory: /root/.cache/pip/wheels/67/b7/d0/a121106e61339eee5ed083bc230b1c8dc422c49a5a28c2addd
Successfully built jellyfish segtok


In [20]:
# YAKE Keyword with 3*3 n-gram
import yake
kw_extractor = yake.KeywordExtractor(lan='en', n=3, dedupLim=0.9, top=10, features=None)

In [21]:
# YAKE Keyword with 2*2 n-gram
import yake
kw_extractor = yake.KeywordExtractor(lan='en', n=2, dedupLim=0.9, top=10, features=None)

In [21]:
# Extract keywords from JD dataset and add them in a new column "YAKE_3"
keyword_list = []
for val in df['clean']:
    keywords = kw_extractor.extract_keywords(val)
    keyword_list.append(keywords)
df['YAKE_3'] = keyword_list

#Apply the same for "YAKE_2" too.
#Add df['YAKE_2'] = keyword_list too

In [22]:
print(keyword_list[0])


[('charitable giving committee', 0.0001815549978237832), ('teamsoft madison areas', 0.0002875543321592835), ('giving committee donated', 0.0003568235385369565), ('participate charitable giving', 0.0004057904289311573), ('giving committee date', 0.0004057904289311573), ('madison areas technology', 0.00042596919973938795), ('madison areas largest', 0.00042596919973938795), ('areas technology leader', 0.0004501888910365571), ('local professionals team', 0.00045812895333495853), ('owners local professionals', 0.0004589495844564971)]


In [23]:
df.head()

,job_description,clean,YAKE_3
0,teamsoft is seeing an it support specialist to...,teamsoft seeing support specialist join client...,"[(charitable giving committee, 0.0001815549978..."
1,the wisconsin state journal is seeking a flexi...,wisconsin state journal seeking flexible motiv...,"[(employerpreemployment drug testing, 0.001766..."
2,report this job about the job depuy synthes co...,report job job depuy synthes companies member ...,"[(depuy synthes companies, 0.00011987891910017..."
3,why join altec? if youâ€™re considering a care...,join altec youâre considering career altec inc...,"[(people greatest strength, 0.0001544430373158..."
4,position id#â 76162 # positionsâ 1 stateâ c...,position idâ 76162 positionsâ 1 stateâ ct city...,"[(assistant store manager, 9.179867706455019e-..."


In [24]:
pip install keybert

     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 211 kB 65.9 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 
     |████████████████████████████████| 2.8 MB 41.4 MB/s 
     |████████████████████████████████| 1.2 MB 27.8 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 3.3 MB 55.4 MB/s 
     |████████████████████████████████| 636 kB 54.2 MB/s 
     |████████████████████████████████| 895 kB 54.8 MB/s 
  Created wheel for keybert: filename=keybert-0.4.0-py3-none-any.whl size=20359 sha256=bb929717e10fcee67f6ffcc49f3512a6c35c969cde410ff7b814e7f37bc4bbfc
  Stored in directory: /root/.cache/pip/wheels/fe/e1/7f/8ee72e851e92320fba5c68d9e6e28d227671876429bc80f420
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=6a5f596d4f80b515a026958eebf85578e8da65a2dbd5659928f5c278a576c2f1
  Stored in directory: /root/.cache/pip/whee

In [25]:
pip install sentence-transformers

In [26]:
#keyBERT with 3*3 n-grams and create a new column "KeyBERT_3" 

from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (3, 3)

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

from sklearn.metrics.pairwise import cosine_similarity

top_n = 10

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
keyword_list_KeyBERT = []
i=0
for val in df['clean']:
  count = CountVectorizer(ngram_range=n_gram_range).fit([val])
  candidates = count.get_feature_names()
  doc_embedding = model.encode([val])
  candidate_embeddings = model.encode(candidates)
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
  keyword_list_KeyBERT.append(keywords)
  i=i+1

In [28]:
df['keyBERT 3_3'] = keyword_list_KeyBERT

In [29]:
df.head()

,job_description,clean,YAKE_3,keyBERT 3_3
0,teamsoft is seeing an it support specialist to...,teamsoft seeing support specialist join client...,"[(charitable giving committee, 0.0001815549978...","[longest running consultant, expertise teamsof..."
1,the wisconsin state journal is seeking a flexi...,wisconsin state journal seeking flexible motiv...,"[(employerpreemployment drug testing, 0.001766...","[reportereditor lead threeperson, threeperson ..."
2,report this job about the job depuy synthes co...,report job job depuy synthes companies member ...,"[(depuy synthes companies, 0.00011987891910017...","[family companies recruiting, orthopedic neuro..."
3,why join altec? if youâ€™re considering a care...,join altec youâre considering career altec inc...,"[(people greatest strength, 0.0001544430373158...","[customer satisfaction everything, recognized ..."
4,position id#â 76162 # positionsâ 1 stateâ c...,position idâ 76162 positionsâ 1 stateâ ct city...,"[(assistant store manager, 9.179867706455019e-...","[store manager associate, store manager accoun..."


In [40]:
# Import CVs
from google.colab import files
uploaded = files.upload()

Saving 1326.docx to 1326.docx


In [31]:
pip install python-docx 

     |████████████████████████████████| 5.6 MB 34.3 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=b6e9597cda5a8df512df4c436724dafa5b6c2fa9687e6b9cef49a35af11b4739
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [41]:
from docx import Document

def getText(filename):
    doc = Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

docxString = getText("1326.docx")

In [33]:
#Add CVs to python objects
CV1 = docxString

In [36]:
#Add CVs to python objects
CV2 = docxString

In [39]:
#Add CVs to python objects
CV3 = docxString

In [42]:
#Add CVs to python objects
CV4 = docxString

In [33]:
print(docxString)

Ravinder Singh
Mobile : 07459557728	   Email : 
LinkedIn : 
Iver, Buckinghamshire

Summary 

Strong analytical skills with the ability to map the given business requirements into required technology components, designs, and workflows.
Knowledge of Data Preparation and Cleaning, Exploratory Data Analysis, Modelling and Data Visualisation.
Business Analyst in the Financial Services Domain Consultant Group with extensive functional knowledge and consulting experience across Commercial Lending, Sovereign Lending.
Technical Skillset: Machine Learning, Python, R, Tableau, Power BI, .Net, SQL/Oracle, MS office tools, Hadoop.

Education and Qualification 

Sept 2020 – Sept 2021	MSc Data Science and Analytics	Brunel University, London
Key modules
Statistics
Machine Learning

Aug 2012 – July 2016		BE Computer Science and Engineering   Chandigarh, India

Certifications
Microsoft Certified: AZ-900 Azure Fundamentals
Microsoft Certified: AI-900 Azure AI Fundamentals
Microsoft Certified: DP-900 Azur

In [43]:
# Lowercase all text CV1
CV1 = CV1.lower()

# Remove Stopwords
from nltk.corpus import stopwords
CV1 = ' '.join([word for word in CV1.split() if word not in (stopwords.words('english'))])

# Remove Punctuations
CV1 = CV1.replace('[^\w\s]','')

In [44]:
# Lowercase all text CV2
CV2 = CV2.lower()

# Remove Stopwords
from nltk.corpus import stopwords
CV2 = ' '.join([word for word in CV2.split() if word not in (stopwords.words('english'))])

# Remove Punctuations
CV2 = CV2.replace('[^\w\s]','')

In [45]:
# Lowercase all text CV3
CV3 = CV3.lower()

# Remove Stopwords
from nltk.corpus import stopwords
CV3 = ' '.join([word for word in CV3.split() if word not in (stopwords.words('english'))])

# Remove Punctuations
CV3 = CV3.replace('[^\w\s]','')

In [46]:
# Lowercase all text CV4
CV4 = CV4.lower()

# Remove Stopwords
from nltk.corpus import stopwords
CV4 = ' '.join([word for word in CV4.split() if word not in (stopwords.words('english'))])

# Remove Punctuations
CV4 = CV4.replace('[^\w\s]','')

In [48]:
CV1

'resume abhishek magotra h.no-632, sector-4, channi himmat, jammu (j&k) email- abhisheksharma268@gmail.com ph no.- 9622225172 career objective:- succeed environment growth excellence earn job provides job satisfaction self development help achieve personal well organizational goals. academic qualification:- major project work:- attended 2 weak survey camp dharamshala,himachal pradesh. achievements: certificate participation painting competition. "inter college cricket tournament" sports meet college. extra curricular activities:- certificate participation college fest techraze’13. participation ptu cricket tournament. training: done 6 month training jammu & kashmir housing board. personal details:- father’s name : mr. ashwani kumar sharma mother’s name : mrs. sunita magotra date birth : 18-march-1993 hobbies & interests : net surfing, listening music personal qualities : hardwork, punctual, adaptable nationality : indian language : hindi, english & dogri. declaration:- hereby solemnly 

In [ ]:
keyword_list_cv = []

In [57]:
#Apply for all CVs
i=0
count = CountVectorizer(ngram_range=n_gram_range).fit([CV4])
candidates = count.get_feature_names()
doc_embedding = model.encode([CV4])
candidate_embeddings = model.encode(candidates)
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords_cv = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keyword_list_cv.append(keywords_cv)


In [58]:
keyword_list_cv[3]

['project proposals assessing',
 '2009 industrial training',
 'planning procurement life',
 'project engineer planning',
 'new project proposals',
 'planning vendor development',
 'towards growth organization',
 'preparation new project',
 'production planning spearheading',
 'engineers preparation new']

In [60]:
for val in keyword_list_cv:
  print(val)

['python project predict', 'data science analytics', 'microsoft certified ai', 'learning python tableau', 'sql database dissertation', 'singh mobile 07459557728', '2021 msc data', '2020 business analyst', 'mobile 07459557728 email', '07459557728 email linkedin']
['meet college extra', 'participation college fest', 'email abhisheksharma268 gmail', 'jammu email abhisheksharma268', 'cricket tournament sports', 'abhisheksharma268 gmail com', 'college fest techraze', 'ptu cricket tournament', 'cricket tournament training', 'college cricket tournament']
['nairobi phone 0724619', 'safaricom box 66827', 'cambridge p2mp solutions', 'latest inventions telecommunication', 'rollout professional ip20c', '726 80100 mombasa', '85575 80100 mombasa', '80100 mombasa phone', '80100 mombasa telephone', 'mombasa phone 0710466547']
['project proposals assessing', '2009 industrial training', 'planning procurement life', 'project engineer planning', 'new project proposals', 'planning vendor development', 'tow

In [61]:
pip install fuzzywuzzy

In [62]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [67]:
# Function to match CV with JD and get matching score for CV and JD
from functools import reduce
List1 = keyword_list[7]
List2 = keyword_list_cv[0]

def matchingLists(List1,List2) :
  score_List = []
  result_list = []
  for i in List1:
    sum=0
    for j in List2:
      score_List.append(fuzz.partial_ratio(i[0],j))
    o = reduce(lambda x,y:x+y, score_List)
    score_List.clear()
    result_list.append(o/10)
    #print(reduce(lambda x,y:x+y, result_list)/10)
  return result_list

d = matchingLists(List1,List2)
print("List of scores after matching each skill from JD with all skills from CV")
print(d)
print("")
print("Overall score of matching a CV with the JD")
print(sum(d)/10)

List of scores after matching each skill from JD with all skills from CV
[28.5, 28.7, 30.9, 31.1, 31.7, 30.9, 36.0, 34.8, 30.6, 32.5]

Overall score of matching a CV with the JD
31.57


In [73]:
# Match multiple CVs
from functools import reduce
List1 = keyword_list[7]
List2 = keyword_list_cv[0]

def matchingLists(List1,List2) :
  score_List = []
  result_list = []
  for i in List1:
    sum=0
    for j in List2:
      score_List.append(fuzz.partial_ratio(i[0],j))
    o = reduce(lambda x,y:x+y, score_List)
    score_List.clear()
    result_list.append(o/10)
    #print(reduce(lambda x,y:x+y, result_list)/10)
  return result_list

d = matchingLists(List1,List2)
print("Overall score of matching a CV1 with the JD")
print(sum(d)/10)

Overall score of matching a CV1 with the JD
31.57


In [ ]:
# Function to match CV with JD and get matching score for CV and JD
# Compare multiple CVs with one JD
from functools import reduce
List1 = keyword_list[6]
ListCV1 = keyword_list_cv[0]
ListCV2 = keyword_list_cv[0]
ListCV3 = keyword_list_cv[0]

def matchingLists(List1,List2) :
  score_List = []
  result_list = []
  for i in List1:
    sum=0
    for j in List2:
      score_List.append(fuzz.partial_ratio(i[0],j))
    o = reduce(lambda x,y:x+y, score_List)
    score_List.clear()
    result_list.append(o/10)
    #print(reduce(lambda x,y:x+y, result_list)/10)
  return result_list

d = matchingLists(List1,List2)
print("List of scores after matching each skill from JD with all skills from CV")
print(d)
print("")
print("Overall score of matching a CV with the JD")
print(sum(d)/10)

In [76]:
pip install datefinder

In [77]:
#Extract Dates from Text
import datefinder

matches = datefinder.find_dates(docxString)
for match in matches:
    print(match)

2013-05-13 00:00:00
2010-12-13 00:00:00
2013-04-13 00:00:00
2021-09-13 00:00:00
2010-07-13 00:00:00
2010-11-13 00:00:00
2021-09-01 00:00:00
2011-09-13 00:00:00
2021-09-01 00:00:00
2009-06-13 00:00:00
2021-12-13 00:00:00
2021-01-13 00:00:00
2021-09-03 00:00:00
2021-09-04 00:00:00
2009-09-13 00:00:00
2009-09-13 00:00:00
2021-09-18 00:00:00
2021-09-16 00:00:00
2021-09-17 00:00:00
2021-09-19 00:00:00
1987-06-23 00:00:00
2021-12-13 00:00:00
